# Билет 38

## Задание 1

## Задание 2

Датасет: Chinook_Sqlite.sqlite.

С помощью кода на питон с испольщованием sqllite3 and SQL решить задачу. Реализовать функции питон: 1. Которая возвращает всех имеющихся артистов 2. Которая по имени артиста возвращает все его альбомы с кол-вом треков в них

In [56]:
import sqlite3

In [57]:
def get_all_artists(cur: sqlite3.Cursor) -> list:
    stmt = 'SELECT * FROM Artist'
    return cur.execute(stmt).fetchall()


def get_artist_albums(cur: sqlite3.Cursor, artist_name: str) -> list:
    stmt = '''
    SELECT AL.AlbumId, AL.Title, COUNT(T.TrackId)
    FROM Album AL
    JOIN Artist AR ON AR.ArtistId = AL.ArtistId
    JOIN Track T ON AL.AlbumId = T.AlbumId
    WHERE AR.Name = ? GROUP BY T.AlbumId;
    '''
    return cur.execute(stmt, [artist_name]).fetchall()

In [58]:
con = sqlite3.connect('data/task_2/Chinook_Sqlite.sqlite')
cur = con.cursor()

In [59]:
get_all_artists(cur)

[(1, 'AC/DC'),
 (2, 'Accept'),
 (3, 'Aerosmith'),
 (4, 'Alanis Morissette'),
 (5, 'Alice In Chains'),
 (6, 'Antônio Carlos Jobim'),
 (7, 'Apocalyptica'),
 (8, 'Audioslave'),
 (9, 'BackBeat'),
 (10, 'Billy Cobham'),
 (11, 'Black Label Society'),
 (12, 'Black Sabbath'),
 (13, 'Body Count'),
 (14, 'Bruce Dickinson'),
 (15, 'Buddy Guy'),
 (16, 'Caetano Veloso'),
 (17, 'Chico Buarque'),
 (18, 'Chico Science & Nação Zumbi'),
 (19, 'Cidade Negra'),
 (20, 'Cláudio Zoli'),
 (21, 'Various Artists'),
 (22, 'Led Zeppelin'),
 (23, 'Frank Zappa & Captain Beefheart'),
 (24, 'Marcos Valle'),
 (25, 'Milton Nascimento & Bebeto'),
 (26, 'Azymuth'),
 (27, 'Gilberto Gil'),
 (28, 'João Gilberto'),
 (29, 'Bebel Gilberto'),
 (30, 'Jorge Vercilo'),
 (31, 'Baby Consuelo'),
 (32, 'Ney Matogrosso'),
 (33, 'Luiz Melodia'),
 (34, 'Nando Reis'),
 (35, 'Pedro Luís & A Parede'),
 (36, 'O Rappa'),
 (37, 'Ed Motta'),
 (38, 'Banda Black Rio'),
 (39, 'Fernanda Porto'),
 (40, 'Os Cariocas'),
 (41, 'Elis Regina'),
 (42, 'Mi

In [60]:
get_artist_albums(cur, 'AC/DC')

[(1, 'For Those About To Rock We Salute You', 10), (4, 'Let There Be Rock', 8)]

## Задание 3

Датасет: movies.txt.

Из названий фильмов, заданных в файле movie.txt, выделить первое слово, второе слово, третье слово и остальные слова. Второе и последуюшие слова выделяются, если они имеются, остальные слова названия включают четвёртое и все последующие слова названия. Словом считается любой выделенный разделителем набор символов, не отностящихся к знакам препинания (отдельно стоящий символ "&" считается свловом). Найти по 2 самых популярных слова, относящихся к каждой из 4х групп. Сформировать из 8 найденных слов список. Распараллелить расчёт при помощи dask.delayed.

In [61]:
import dask
from collections import defaultdict
import string

In [62]:
@dask.delayed
def get_groups():
    with open('data/task_3/movies.txt') as f:
        punctuation = set(string.punctuation) - {'&'}
        word_groups = defaultdict(lambda: defaultdict(int))
        for line in f:
            words = [w for w in line.split() if w not in punctuation]
            if len(words) > 0:
                word_groups['first'][words[0]] += 1
            if len(words) > 1:
                word_groups['second'][words[0]] += 1
            if len(words) > 2:
                word_groups['third'][words[2]] += 1
            if len(words) > 3:
                for w in words[3:]:
                    word_groups['fourth'][w] += 1

        most_common_2 = {}
        for key, value in word_groups.items():
            most_common_2[key] = sorted(value.items(), key=lambda x: x[1], reverse=True)[:2]
    return most_common_2

In [63]:
groups = get_groups().compute()
[w for group in groups.values() for w, _ in group]

['The', 'A', 'The', 'A', 'of', 'the', 'the', 'of']